##  Template for analysis notebooks

### Imports

In [2]:
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import traceback
import pandas_datareader.data as pdr


### Run this cell if you want to suppress warnings.  This is useful if the warnings print local information that you don't want to show the public but you still want to save this notebook with its output.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### Instantiate an instance of pg_pandas

In [6]:
pga = db_info.get_db_info()
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'


  sec_db


### Define a generalized plotly chart method

In [7]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdvals,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    return fig


### Define method to access x number of successive commodity contracts from ```{futtab}```

In [ ]:
def consecutive_contracts(commod,num_contracts,first_settle_date_yyyymmdd):
    sql = f"""
    select distinct symbol from {futtab} 
    where settle_date>={first_settle_date_yyyymmdd} and 
    symbol ~* '^{commod}';
    """
    symbols = pga.get_sql(sql).symbol.values
    symbols_ppyymm = sorted([f'{c[:(len(c)-3)]}' + f'{c[-2:]}' + f'{c[2]}' for c in symbols])
    return symbols_ppyymm[:num_contracts]

consecutive_contracts('CL',12,20180401)